Lambda School Data Science

*Unit 2, Sprint 2, Module 1*

---

# Decision Trees

## Assignment
- $\checkmark$ [Sign up for a Kaggle account](https://www.kaggle.com/), if you don’t already have one. Go to our Kaggle InClass competition website. You will be given the URL in Slack. Go to the Rules page. Accept the rules of the competition.
- $\checkmark$ Do train/validate/test split with the Tanzania Waterpumps data.
- $\checkmark$ Begin with baselines for classification.
- $\checkmark$ Select features. Use a scikit-learn pipeline to encode categoricals, impute missing values, and fit a decision tree classifier.
- [ ] Get your validation accuracy score.
- [ ] Get and plot your feature importances.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


## Stretch Goals

### Reading

- A Visual Introduction to Machine Learning
  - [Part 1: A Decision Tree](http://www.r2d3.us/visual-intro-to-machine-learning-part-1/)
  - [Part 2: Bias and Variance](http://www.r2d3.us/visual-intro-to-machine-learning-part-2/)
- [Decision Trees: Advantages & Disadvantages](https://christophm.github.io/interpretable-ml-book/tree.html#advantages-2)
- [How a Russian mathematician constructed a decision tree — by hand — to solve a medical problem](http://fastml.com/how-a-russian-mathematician-constructed-a-decision-tree-by-hand-to-solve-a-medical-problem/)
- [How decision trees work](https://brohrer.github.io/how_decision_trees_work.html)
- [Let’s Write a Decision Tree Classifier from Scratch](https://www.youtube.com/watch?v=LDRbO9a6XPU) — _Don’t worry about understanding the code, just get introduced to the concepts. This 10 minute video has excellent diagrams and explanations._
- [Random Forests for Complete Beginners: The definitive guide to Random Forests and Decision Trees](https://victorzhou.com/blog/intro-to-random-forests/)


### Doing
- [ ] Add your own stretch goal(s) !
- [ ] Define a function to wrangle train, validate, and test sets in the same way. Clean outliers and engineer features. (For example, [what columns have zeros and shouldn't?](https://github.com/Quartz/bad-data-guide#zeros-replace-missing-values) What columns are duplicates, or nearly duplicates? Can you extract the year from date_recorded? Can you engineer new features, such as the number of years from waterpump construction to waterpump inspection?)
- [ ] Try other [scikit-learn imputers](https://scikit-learn.org/stable/modules/impute.html).
- [ ] Make exploratory visualizations and share on Slack.


#### Exploratory visualizations

Visualize the relationships between feature(s) and target. I recommend you do this with your training set, after splitting your data. 

For this problem, you may want to create a new column to represent the target as a number, 0 or 1. For example:

```python
train['functional'] = (train['status_group']=='functional').astype(int)
```



You can try [Seaborn "Categorical estimate" plots](https://seaborn.pydata.org/tutorial/categorical.html) for features with reasonably few unique values. (With too many unique values, the plot is unreadable.)

- Categorical features. (If there are too many unique values, you can replace less frequent values with "OTHER.")
- Numeric features. (If there are too many unique values, you can [bin with pandas cut / qcut functions](https://pandas.pydata.org/pandas-docs/stable/getting_started/basics.html?highlight=qcut#discretization-and-quantiling).)

You can try [Seaborn linear model plots](https://seaborn.pydata.org/tutorial/regression.html) with numeric features. For this classification problem, you may want to use the parameter `logistic=True`, but it can be slow.

You do _not_ need to use Seaborn, but it's nice because it includes confidence intervals to visualize uncertainty.

#### High-cardinality categoricals

This code from a previous assignment demonstrates how to replace less frequent values with 'OTHER'

```python
# Reduce cardinality for NEIGHBORHOOD feature ...

# Get a list of the top 10 neighborhoods
top10 = train['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10,
# replace the neighborhood with 'OTHER'
train.loc[~train['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'
test.loc[~test['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'
```


In [1]:
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

     |████████████████████████████████| 102kB 2.4MB/s 


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

train, val = train_test_split(train, train_size= 0.75, test_size= 0.25)

train.shape, val.shape, test.shape

((44550, 41), (14850, 41), (14358, 40))

In [0]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

In [4]:
target = 'status_group'

train_features = train.drop(columns=[target, 'id'])
numeric_features = train_features.select_dtypes(include='number').columns.tolist()
cardinality = train_features.select_dtypes(exclude='number').nunique()
categorical_features = cardinality[cardinality <= 25].index.tolist()

features = numeric_features + categorical_features

X_train = train[features]
y_train = train[target]

X_val = val[features]
y_val = val[target]

X_test = test[features]

pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True), 
    SimpleImputer(strategy='mean'), 
    StandardScaler(), 
    LogisticRegression(multi_class='auto', solver='lbfgs', n_jobs=-1)
)
pipeline.fit(X_train, y_train)

# Score on train, val
print('Train Accuracy', pipeline.score(X_train, y_train))
print('Validation Accuracy', pipeline.score(X_val, y_val))

# Predict on test
y_pred = pipeline.predict(X_test)

Train Accuracy 0.7359820426487094
Validation Accuracy 0.7276094276094276


In [5]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True), 
    SimpleImputer(strategy='mean'), 
    DecisionTreeClassifier()
)

# Fit on train
pipeline.fit(X_train, y_train)

# Score on train, val
print('Train Accuracy', pipeline.score(X_train, y_train))
print('Validation Accuracy', pipeline.score(X_val, y_val))

y_pred = pipeline.predict(X_test)

Train Accuracy 0.9956228956228956
Validation Accuracy 0.7534006734006734


In [0]:
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('submission.csv', index=False)

In [0]:
import sklearn.tree

In [8]:
pipeline0 = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True), 
    SimpleImputer(strategy='mean'), 
    sklearn.tree.ExtraTreeClassifier(criterion='MAE')
)

pipeline0.fit(X_train, y_train)

print('Train Accuracy', pipeline0.score(X_train, y_train))
print('Validation Accuracy', pipeline0.score(X_val, y_val))

y_pred = pipeline0.predict(X_test)

KeyError: ignored

In [0]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler




def test(train_x, train_y, val_x, val_y, test_x, parameters):
  pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True), 
    SimpleImputer(strategy='mean'), 
    DecisionTreeClassifier(criterion="entropy",
                           max_depth= parameters[0],
                           max_features= parameters[1],
                           max_leaf_nodes= parameters[2],
                           )
  )

  pipeline.fit(train_x, train_y)

  a = ['Train Accuracy: ',pipeline.score(train_x, train_y)]
  b = ['Validation Accuracy: ',pipeline.score(val_x, val_y)]

  c = pipeline.predict(test_x)
  return [a,b,c]

In [0]:
a = {}
range_length = 20

for x in range(5,range_length):
  for y in range(5,range_length):
    for z in range(5,range_length):
      a[x,y,z] = test(X_train, y_train, X_val, y_val, X_test, [x,y,z])
      print('z:\t'+str(z/range_length))
    print('y:\t'+str(y/range_length))
  print('x:\t'+str(x/range_length))

#a = test(X_train, y_train, X_val, y_val, X_test, max_depth= 7)

z:	0.25
z:	0.3
z:	0.35
z:	0.4
z:	0.45
z:	0.5
z:	0.55
z:	0.6
z:	0.65
z:	0.7
z:	0.75
z:	0.8
z:	0.85
z:	0.9
z:	0.95
y:	0.25
z:	0.25
z:	0.3
z:	0.35
z:	0.4
z:	0.45
z:	0.5
z:	0.55
z:	0.6
z:	0.65
z:	0.7
z:	0.75
z:	0.8
z:	0.85
z:	0.9
z:	0.95
y:	0.3
z:	0.25
z:	0.3
z:	0.35
z:	0.4
z:	0.45
z:	0.5
z:	0.55
z:	0.6
z:	0.65
z:	0.7
z:	0.75
z:	0.8
z:	0.85
z:	0.9
z:	0.95
y:	0.35
z:	0.25
z:	0.3
z:	0.35
z:	0.4
z:	0.45
z:	0.5
z:	0.55
z:	0.6
z:	0.65
z:	0.7
z:	0.75
z:	0.8
z:	0.85
z:	0.9
z:	0.95
y:	0.4
z:	0.25
z:	0.3
z:	0.35
z:	0.4
z:	0.45
z:	0.5
z:	0.55
z:	0.6
z:	0.65
z:	0.7
z:	0.75
z:	0.8
z:	0.85
z:	0.9
z:	0.95
y:	0.45
z:	0.25
z:	0.3
z:	0.35
z:	0.4
z:	0.45
z:	0.5
z:	0.55
z:	0.6
z:	0.65
z:	0.7
z:	0.75
z:	0.8
z:	0.85
z:	0.9
z:	0.95
y:	0.5
z:	0.25
z:	0.3
z:	0.35
z:	0.4
z:	0.45
z:	0.5
z:	0.55
z:	0.6
z:	0.65
z:	0.7
z:	0.75
z:	0.8
z:	0.85
z:	0.9
z:	0.95
y:	0.55
z:	0.25
z:	0.3
z:	0.35
z:	0.4
z:	0.45
z:	0.5
z:	0.55
z:	0.6
z:	0.65
z:	0.7
z:	0.75
z:	0.8
z:	0.85
z:	0.9
z:	0.95
y:	0.6
z:	0.25
z:	0.3
z:	0.35
z:	0.4
z:	0.4

In [0]:
import matplotlib.pyplot as plt

In [0]:
a

In [0]:
x_s = [x for x in a.keys()]
y_s = [y[1] for y in a.values()]

plt.scatter(x_s, y_s)

In [0]:
from google.colab import files
files.download('submission.csv')